In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [3]:
spark=SparkSession.builder\
        .appName('MySparkApp')\
        .master('local[*]')\
        .getOrCreate()
spark.version

'4.0.1'

In [13]:
users= [
    (5, "Smart Crock Pot", "2021-09-18", 698882),
    (6, "Smart Lock", "2021-09-14", 11487),
    (6, "Smart Thermostat", "2021-09-10", 674762),
    (8, "Smart Light Strip", "2021-09-29", 630773),
    (4, "Smart Cat Feeder", "2021-09-02", 693545),
    (4, "Smart Bed", "2021-09-13", 170249)
]

In [14]:
users_df=spark.createDataFrame(users,['user_id','item','created_at','amount'])
users_df.show()

+-------+-----------------+----------+------+
|user_id|             item|created_at|amount|
+-------+-----------------+----------+------+
|      5|  Smart Crock Pot|2021-09-18|698882|
|      6|       Smart Lock|2021-09-14| 11487|
|      6| Smart Thermostat|2021-09-10|674762|
|      8|Smart Light Strip|2021-09-29|630773|
|      4| Smart Cat Feeder|2021-09-02|693545|
|      4|        Smart Bed|2021-09-13|170249|
+-------+-----------------+----------+------+



In [15]:
w=Window.partitionBy('user_id').orderBy(col('created_at'))
users_df=users_df.withColumn('next_purchase_date',lead('created_at').over(w))
users_df.show()

+-------+-----------------+----------+------+------------------+
|user_id|             item|created_at|amount|next_purchase_date|
+-------+-----------------+----------+------+------------------+
|      4| Smart Cat Feeder|2021-09-02|693545|        2021-09-13|
|      4|        Smart Bed|2021-09-13|170249|              NULL|
|      5|  Smart Crock Pot|2021-09-18|698882|              NULL|
|      6| Smart Thermostat|2021-09-10|674762|        2021-09-14|
|      6|       Smart Lock|2021-09-14| 11487|              NULL|
|      8|Smart Light Strip|2021-09-29|630773|              NULL|
+-------+-----------------+----------+------+------------------+



In [16]:
users_df=users_df[date_diff(users_df['next_purchase_date'],users_df['created_at'])<=7][['user_id']]
users_df.show()

+-------+
|user_id|
+-------+
|      6|
+-------+



25/10/11 01:13:50 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 381815 ms exceeds timeout 120000 ms
25/10/11 01:13:50 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/11 01:19:51 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$